<a href="https://colab.research.google.com/github/as2c/MLP---Cosmology/blob/main/density_contrast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
!git clone https://github.com/jveitchmichaelis/pygadgetreader

Cloning into 'pygadgetreader'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 236 (delta 6), reused 236 (delta 6), pack-reused 0
Receiving objects: 100% (236/236), 749.63 KiB | 6.88 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
%cd /content/pygadgetreader
!python setup.py build
!python setup.py install

/content/pygadgetreader
running build
running build_py
running install
running bdist_egg
running egg_info
writing pyGadgetReader.egg-info/PKG-INFO
writing dependency_links to pyGadgetReader.egg-info/dependency_links.txt
writing requirements to pyGadgetReader.egg-info/requires.txt
writing top-level names to pyGadgetReader.egg-info/top_level.txt
adding license file 'LICENSE'
writing manifest file 'pyGadgetReader.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/pygadgetreader
copying build/lib/pygadgetreader/__init__.py -> build/bdist.linux-x86_64/egg/pygadgetreader
creating build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readpstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readrockstar.py -> build/bdist.linux-x86_64/egg/readgadget
copying build/lib/readgadget/readgadget.py -> build/bdist.linux-x8

In [ ]:
from pygadgetreader import *

In [ ]:
vel = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','vel','dm')
vx = vel[:,0].tolist()
vy = vel[:,1].tolist()
vz = vel[:,2].tolist()
mass = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','mass','dm').tolist()
particle_id = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pid','dm').tolist()
position = readsnap('/content/drive/MyDrive/ML/n1lcdmML_snap000_cdm','pos','dm')
x = position[:,0].tolist()
y = position[:,1].tolist()
z = position[:,2].tolist()
data = {'particle_id': particle_id,
        'x': x,
        'y': y,
        'z': z,
        'vx': vx,
        'vy': vy,
        'vz': vz,
        'mass': mass}
df = pd.DataFrame(data)

Returning DM    Velocities
Returning DM    Mass
Returning DM    Particle IDs
Returning DM    Positions


In [ ]:
df.head()

,particle_id,x,y,z,vx,vy,vz,mass
0,0,192.691193,231.858017,225.984268,-659.554321,-419.040802,-489.896637,2.02478
1,7,809.101929,210.431351,214.217163,-702.608582,-530.673584,-534.257935,2.02478
2,56,1437.270874,198.266098,201.348267,-689.156860,-608.542847,-615.787476,2.02478
3,63,2043.859375,196.952347,187.470413,-797.107727,-646.467102,-686.565735,2.02478
4,64,2664.605225,204.039200,170.532455,-818.485474,-626.312866,-768.481689,2.02478


In [ ]:
import pickle5 as pickle
df = pd.read_pickle("/content/drive/MyDrive/LCDM_Data/lcdm_z=100.gz")

ModuleNotFoundError: ignored

In [ ]:
%cd https://drive.google.com/drive/folders/1Hmbt7aDEB8eJJH-9BCVqBCZS1tlo3WFz?usp=sharing

[Errno 2] No such file or directory: 'https://drive.google.com/drive/folders/1Hmbt7aDEB8eJJH-9BCVqBCZS1tlo3WFz?usp=sharing'
/content


In [ ]:
df.drop(['mass'], axis = 1, inplace = True)

In [ ]:
df.insert(1, 'in-halo', 0)

The following line of code has to be changed, since we only assign a random value of 0 or 1 for each particle's in-halo value.

In [ ]:
df['in-halo'] = np.random.choice([0, 1], df.shape[0])
# df.head(10)

In [ ]:
df = df.sort_values( by =['x'])

In [ ]:
data = df.copy(deep = True)
data.x = data.x / 1000
data.y = df.y / 1000
data.z = df.z / 1000
print(data.x.max())
print(data.y.max())
print(data.z.max())

159.846765625
159.85021875
159.824359375


In [ ]:
box_size = 159
boundary = 20

d = data[data.x > boundary]

d = d[data.x < box_size - boundary]

d = d[data.y > boundary]

d = d[data.y < box_size - boundary]

d = d[data.z > boundary]

d = d[data.z < box_size - boundary]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [ ]:
def box_mass(r_cutoff, df, d):
    m = len(df)
    mass = pd.DataFrame(np.zeros(m),columns=['den_contrast'])
    for i in range(m):
        wx=d.iloc[i]['x']
        wy=d.iloc[i]['y']
        wz=d.iloc[i]['z']
        ix = np.searchsorted(df[:,2] , wx-r_cutof)
        jx = np.searchsorted(df[:,2] , wx+r_cutof)
        temp = np.where((df[ix:jx,3] < wy+r_cutoff) & (df[ix:jx,3] > wy-r_cutoff) & (df[ix:jx,4] < wz+r_cutoff) & (df[ix:jx,4] > wz-r_cutoff) )
    
        mass.den_contrast[i] = len(temp[0])
        print(i)
    return mass

In [ ]:
n = 100000

sample = d.sample(n, random_state = 1)
sample_density = sample[['particle_id', 'in-halo']]
sample_density.reset_index(drop = True, inplace = True)
sample_density.head()

,particle_id,in-halo
0,6734555,0
1,12718733,1
2,6470459,0
3,12098249,1
4,4799280,1


In [ ]:
# print(data)

          particle_id  in-halo           x  ...           vx          vy          vz
2573056       3702803        0    0.104388  ... -1147.799561 -421.298645 -159.373352
2630144       4227097        1    0.106355  ... -1127.608154 -401.552826 -111.468544
11749888     10133520        0    0.107426  ... -1146.397827  135.866959 -227.624420
11750144     10133523        0    0.108411  ... -1141.314331   48.016113 -231.397110
2654464       4489219        0    0.108463  ... -1133.096680 -515.861816 -212.447449
...               ...      ...         ...  ...          ...         ...         ...
10339839     15605748        1  159.840266  ...   844.709229 -175.208862 -443.182190
10348031     15605749        0  159.842984  ...   859.852356  -94.854485 -464.136230
10340351     15605740        1  159.844250  ...   869.186951 -289.623627 -300.264465
10348287     15605742        0  159.845828  ...   885.247620  -69.581429 -391.406372
10340095     15605743        0  159.846766  ...   881.997192 -113

In [ ]:
data = data.to_numpy()

In [ ]:
rc = np.array([])
for i in range(8, 11, 1):
  r = 2 + 1 * i
  rc = np.append(rc, r)
  density = box_mass(r , data, sample)
  density = density.div((2*r)**3)
  sample_density = pd.concat([sample_density, density], axis = 1)
sample_density.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


KeyboardInterrupt: ignored